# Setup

In [21]:
# import your standard packages and parameters
%run ../pkgs.py
%run ../pars.py

# import your local functions
sys.path.insert(1, '../')
from local_functions import *
from MOM6_functions import *

# make sure the figures plot inline rather than at the end
%matplotlib inline

Default libraries loaded.


# Paths and Fns

In [22]:
inpath = '/scratch/gpfs2/GEOCLIM/LRGROUP/Liao/MOM6-regional-0522-2021/ice_ocean_SIS2/regional_IndianOcean_control_config_025_correctaxis/'
outpath = '../../data/'

outfn = '3d_var_regional_processed.nc'

In [23]:
fns = sorted(glob.glob(inpath +'*dynamics3d_monthly*.nc')) # sorted() makes sure the files are sorted in time
ds_in = xr.open_dataset(fns[0])
ds_in

<xarray.Dataset>
Dimensions:       (nv: 2, time: 12, xh: 428, xq: 429, yh: 256, yq: 257, z_i: 36, z_l: 35)
Coordinates:
  * xh            (xh) float64 29.38 29.62 29.88 30.12 ... 135.6 135.9 136.1
  * yh            (yh) float64 -30.46 -30.24 -30.02 -29.81 ... 29.81 30.02 30.24
  * time          (time) object 2009-01-16 12:00:00 ... 2009-12-16 12:00:00
  * nv            (nv) float64 1.0 2.0
  * xq            (xq) float64 29.25 29.5 29.75 30.0 ... 135.5 135.8 136.0 136.2
  * yq            (yq) float64 -30.56 -30.35 -30.13 -29.92 ... 29.92 30.13 30.35
  * z_l           (z_l) float64 2.5 10.0 20.0 32.5 ... 5.5e+03 6e+03 6.5e+03
  * z_i           (z_i) float64 0.0 5.0 15.0 25.0 ... 5.75e+03 6.25e+03 6.75e+03
Data variables:
    geolat        (yh, xh) float32 ...
    geolon        (yh, xh) float32 ...
    geolat_c      (yq, xq) float32 ...
    geolon_c      (yq, xq) float32 ...
    geolat_u      (yh, xq) float32 ...
    geolon_u      (yh, xq) float32 ...
    geolat_v      (yq, xh) float32 ..

# Get Regional Data

In [24]:
fns = sorted(glob.glob(inpath +'*dynamics3d_monthly*.nc')) # sorted() makes sure the files are sorted in time

# merge all ssh into one file
cnt = 0;
for ff, fn in enumerate(fns[:-1]):
    ds_in = xr.open_dataset(fn)
    
    print('processing: ', fn[-7:-3])
    
    if ff == 0:
        # intialize
        temp = ds_in.temp
        salt = ds_in.salt
        u = ds_in.u
        v = ds_in.v
        Kd_interface = ds_in.Kd_interface 
        rhopot0 = ds_in.rhopot0
#         ea = ds_in.ea
#         eb = ds_in.eb
        umo = ds_in.umo
        vmo = ds_in.vmo
#         N2 = ds_in.N2
        
        lat = np.array(ds_in.geolat)
        lon = np.array(ds_in.geolon)
        lat_u = np.array(ds_in.geolat_u)
        lon_u = np.array(ds_in.geolon_u)
        lat_v = np.array(ds_in.geolat_v)
        lon_v = np.array(ds_in.geolon_v)
    else:
        # add on current time step along time dimension
        temp = xr.concat((temp,ds_in.temp),dim = 'time')
        salt = xr.concat((salt,ds_in.salt),dim = 'time')
        u = xr.concat((u,ds_in.u),dim = 'time')
        v = xr.concat((v,ds_in.v),dim = 'time')
        Kd_interface = xr.concat((Kd_interface,ds_in.Kd_interface),dim = 'time')
#         ea = xr.concat((ea,ds_in.ea),dim = 'time')
#         eb = xr.concat((eb,ds_in.eb),dim = 'time')
        umo = xr.concat((umo,ds_in.umo),dim = 'time')
        vmo = xr.concat((vmo,ds_in.vmo),dim = 'time')
#         N2 = xr.concat((N2,ds_in.N2),dim = 'time')
        
    cnt = cnt + ds_in.time.shape[0]

print('temp final shape',temp.shape)

processing:  2009
processing:  2010
processing:  2011
processing:  2012
processing:  2013
processing:  2014
processing:  2015
processing:  2016
processing:  2017
temp final shape (108, 35, 256, 428)


In [25]:
# convert to xarray dataset
ds_out = xr.Dataset()

dims = ['time','z_l','lat', 'lon']
coords = [temp.time,temp.z_l,lat[:,0],lon[0,:]]

dims_u = ['time','z_l','lat_u', 'lon_u']
coords_u = [temp.time,temp.z_l,lat_u[:,0],lon_u[0,:]]

dims_v = ['time','z_l','lat_v', 'lon_v']
coords_v = [temp.time,temp.z_l,lat_v[:,0],lon_v[0,:]]

dims_i = ['time','z_i','lat', 'lon']
coords_i = [temp.time,Kd_interface.z_i,lat[:,0],lon[0,:]]

# add variables to dataset
ds_out['temp']=xr.DataArray(np.array(temp),dims = dims,coords =coords)
ds_out['salt']=xr.DataArray(np.array(salt),dims = dims,coords =coords)
ds_out['u']=xr.DataArray(np.array(u),dims = dims_u,coords =coords_u)
ds_out['v']=xr.DataArray(np.array(v),dims = dims_v,coords =coords_v)
ds_out['Kd_interface']=xr.DataArray(np.array(Kd_interface),dims = dims_i,coords =coords_i)
# ds_out['ea']=xr.DataArray(np.array(ea),dims = dims,coords =coords)
# ds_out['eb']=xr.DataArray(np.array(eb),dims = dims,coords =coords)
ds_out['umo']=xr.DataArray(np.array(umo),dims = dims_u,coords =coords_u)
ds_out['vmo']=xr.DataArray(np.array(vmo),dims = dims_v,coords =coords_v)
# ds_out['N2']=xr.DataArray(np.array(N2),dims = dims_i,coords =coords_i)

# convert to datetime format
datetimeindex = ds_out.indexes['time'].to_datetimeindex()
ds_out['time'] = datetimeindex
ds_out


/home/jennap/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'julian', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.


<xarray.Dataset>
Dimensions:       (lat: 256, lat_u: 256, lat_v: 257, lon: 428, lon_u: 429, lon_v: 428, time: 108, z_i: 36, z_l: 35)
Coordinates:
  * time          (time) datetime64[ns] 2009-01-16T12:00:00 ... 2017-12-16T12:00:00
  * z_l           (z_l) float64 2.5 10.0 20.0 32.5 ... 5.5e+03 6e+03 6.5e+03
  * lat           (lat) float32 -30.455408 -30.239664 ... 30.023445 30.239664
  * lon           (lon) float32 29.375 29.625 29.875 ... 135.625 135.875 136.125
  * lat_u         (lat_u) float32 -30.455408 -30.239664 ... 30.023445 30.239664
  * lon_u         (lon_u) float32 29.25 29.5 29.75 30.0 ... 135.75 136.0 136.25
  * lat_v         (lat_v) float32 -30.563103 -30.347595 ... 30.131615 30.347595
  * lon_v         (lon_v) float32 29.375 29.625 29.875 ... 135.875 136.125
  * z_i           (z_i) float64 0.0 5.0 15.0 25.0 ... 5.75e+03 6.25e+03 6.75e+03
Data variables:
    temp          (time, z_l, lat, lon) float32 nan nan nan nan ... nan nan nan
    salt          (time, z_l, lat, lon) fl

# Save

In [26]:

# delete if already present
if os.path.isfile(outpath + outfn):
    os.remove(outpath + outfn)

ds_out.to_netcdf(outpath + outfn,mode='w',format = "NETCDF4")

ds_out

<xarray.Dataset>
Dimensions:       (lat: 256, lat_u: 256, lat_v: 257, lon: 428, lon_u: 429, lon_v: 428, time: 108, z_i: 36, z_l: 35)
Coordinates:
  * time          (time) datetime64[ns] 2009-01-16T12:00:00 ... 2017-12-16T12:00:00
  * z_l           (z_l) float64 2.5 10.0 20.0 32.5 ... 5.5e+03 6e+03 6.5e+03
  * lat           (lat) float32 -30.455408 -30.239664 ... 30.023445 30.239664
  * lon           (lon) float32 29.375 29.625 29.875 ... 135.625 135.875 136.125
  * lat_u         (lat_u) float32 -30.455408 -30.239664 ... 30.023445 30.239664
  * lon_u         (lon_u) float32 29.25 29.5 29.75 30.0 ... 135.75 136.0 136.25
  * lat_v         (lat_v) float32 -30.563103 -30.347595 ... 30.131615 30.347595
  * lon_v         (lon_v) float32 29.375 29.625 29.875 ... 135.875 136.125
  * z_i           (z_i) float64 0.0 5.0 15.0 25.0 ... 5.75e+03 6.25e+03 6.75e+03
Data variables:
    temp          (time, z_l, lat, lon) float32 nan nan nan nan ... nan nan nan
    salt          (time, z_l, lat, lon) fl